In [23]:
import os
import json
import pandas as pd
import traceback
from dotenv import load_dotenv, find_dotenv
from tqdm import tqdm_notebook
from langchain_community.llms import HuggingFaceEndpoint, HuggingFaceTextGenInference
from langchain.prompts import PromptTemplate
from langchain.chains import SequentialChain
from langchain.chains import LLMChain
from langchain.callbacks import get_openai_callback

load_dotenv(find_dotenv())
FACE_API = os.getenv('HUGGING_FACE_API')

In [72]:
NUMBER = 5
SUBJECT = 'machine learning'
TONE = 'simple'
RESPNSE_JSON = {
    '1': {
        'mcq': 'multiple choice question',
        'options': {
            'a': 'choice here',
            'b': 'choice here',
            'c': 'choice here',
            'd': 'choice here'
        },
        'correct answer': 'option',
    },
    '2': {
        'mcq': 'multiple choice question',
        'options': {
            'a': 'choice here',
            'b': 'choice here',
            'c': 'choice here',
            'd': 'choice here'
        },
        'correct answer': 'option',
    },
    '3': {
        'mcq': 'multiple choice question',
        'options': {
            'a': 'choice here',
            'b': 'choice here',
            'c': 'choice here',
            'd': 'choice here'
        },
        'correct answer': 'correct option',
    },
}

In [73]:
llm = HuggingFaceEndpoint(
    repo_id='mistralai/Mistral-7B-Instruct-v0.2',
    task='question-answering',
    max_new_tokens=512,
    top_k=50,
    temperature=0.1,
    repetition_penalty=1.03,
    huggingfacehub_api_token=FACE_API
)

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to C:\Users\PC\.cache\huggingface\token
Login successful


In [145]:
TEMPLATE = """
Text: {text}
    You are expert MCQ maker. Given the above text, it is your job to create a quiz of {number} \
    multiple choice questions for {subject} students in {tone} tone.
    Make sure the questions are not repeated and check all the questions to be conforming \
    the text as well.
    Make sure to formate your respose like dictionary don't include other text because I cannot convet it into json and use it as a guide. \
    The response quiz you give me must be converted into JSON. Ensure to make {number} MCQs.
    {response_json}
    """

quiz_generation_prompt = PromptTemplate(
    template=TEMPLATE,
    input_variables=['text', 'number', 'subject', 'tone', 'response_json'])

In [146]:
quiz_chain = LLMChain(llm=llm, prompt=quiz_generation_prompt,
                      output_key='quiz', verbose=True)

In [147]:
TEMPLATE2 = """
Your are an expert english grammarian and writer. Given a Multiple Choice \
Quiz for {subject} students. You need to evaluate the complexity of the \
questions and give a complexity analysis of the quiz. Only use at max 50 words for complexity \
if the quiz is not at per with the cognitive and analytical abilities of the students, \
update the quiz quetstions which needs to be changed update the tone such that it \
perfectly fits the students ability.
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz.
"""

quiz_evaluation_prompt = PromptTemplate(
    input_variables=['subject', 'quiz'],
    template=TEMPLATE2
)

review_chain = LLMChain(
    llm=llm,
    prompt=quiz_evaluation_prompt,
    output_key='review',
    verbose=True
)

In [148]:
generate_evaluate_chain = SequentialChain(
    chains=[quiz_chain, review_chain],
    input_variables=['text', 'number', 'subject', 'tone', 'response_json'],
    output_variables=['quiz', 'review'],
    verbose=True
)

In [149]:
with open("../data.txt", 'r') as file:
    TEXT = file.read()
print(TEXT)

json.dumps(TEXT)

Since deep learning and machine learning tend to be used interchangeably, it’s worth noting the nuances between the two. Machine learning, deep learning, and neural networks are all sub-fields of artificial intelligence. However, neural networks is actually a sub-field of machine learning, and deep learning is a sub-field of neural networks.

The way in which deep learning and machine learning differ is in how each algorithm learns. "Deep" machine learning can use labeled datasets, also known as supervised learning, to inform its algorithm, but it doesn’t necessarily require a labeled dataset. The deep learning process can ingest unstructured data in its raw form (e.g., text or images), and it can automatically determine the set of features which distinguish different categories of data from one another. This eliminates some of the human intervention required and enables the use of large amounts of data. You can think of deep learning as "scalable machine learning" as Lex Fridman notes

'"Since deep learning and machine learning tend to be used interchangeably, it\\u2019s worth noting the nuances between the two. Machine learning, deep learning, and neural networks are all sub-fields of artificial intelligence. However, neural networks is actually a sub-field of machine learning, and deep learning is a sub-field of neural networks.\\n\\nThe way in which deep learning and machine learning differ is in how each algorithm learns. \\"Deep\\" machine learning can use labeled datasets, also known as supervised learning, to inform its algorithm, but it doesn\\u2019t necessarily require a labeled dataset. The deep learning process can ingest unstructured data in its raw form (e.g., text or images), and it can automatically determine the set of features which distinguish different categories of data from one another. This eliminates some of the human intervention required and enables the use of large amounts of data. You can think of deep learning as \\"scalable machine learni

In [150]:
with get_openai_callback() as cb:
    response = generate_evaluate_chain(
        {'text': TEXT,
         'number': NUMBER,
         'subject': SUBJECT,
         'tone': TONE,
         'response_json': json.dumps(RESPNSE_JSON)}
    )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text: Since deep learning and machine learning tend to be used interchangeably, it’s worth noting the nuances between the two. Machine learning, deep learning, and neural networks are all sub-fields of artificial intelligence. However, neural networks is actually a sub-field of machine learning, and deep learning is a sub-field of neural networks.

The way in which deep learning and machine learning differ is in how each algorithm learns. "Deep" machine learning can use labeled datasets, also known as supervised learning, to inform its algorithm, but it doesn’t necessarily require a labeled dataset. The deep learning process can ingest unstructured data in its raw form (e.g., text or images), and it can automatically determine the set of features which distinguish different categories of data from one another. This eliminates some of the human intervention required and enables the use

In [151]:
print(f"Total Tokens: {cb.total_tokens}")
print(f"Prompt Tokens: {cb.prompt_tokens}")
print(f"Completion Tokens: {cb.completion_tokens}")
print(f"Total Cost: {cb.total_cost}")

Total Tokens: 0
Prompt Tokens: 0
Completion Tokens: 0
Total Cost: 0.0


In [196]:
quiz = response.get('quiz')[78:]
quiz = json.loads(quiz)

{'1': {'mcq': 'Which sub-field of artificial intelligence is deep learning a part of?',
  'options': {'a': 'Machine learning',
   'b': 'Neural networks',
   'c': 'Deep learning',
   'd': 'Computer vision'},
  'correct answer': 'a'},
 '2': {'mcq': 'Which type of machine learning is deep learning a sub-field of?',
  'options': {'a': 'Supervised learning',
   'b': 'Unsupervised learning',
   'c': 'Reinforcement learning',
   'd': 'Deep learning'},
  'correct answer': 'a'},
 '3': {'mcq': 'What is the difference between machine learning and deep learning?',
  'options': {'a': 'Machine learning uses labeled datasets, deep learning does not.',
   'b': 'Deep learning requires human intervention, machine learning does not.',
   'c': 'Deep learning can only be used for structured data, machine learning can handle unstructured data.',
   'd': 'Deep learning is a sub-field of machine learning, machine learning is a sub-field of deep learning.'},
  'correct answer': 'a'},
 '4': {'mcq': 'Which type 

In [ ]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [f"{option}: {option_value}" for option, option_value in value["options"].items()]
    )
    correct = value["correct answer"]
    quiz_table_data.append({"MCQ": mcq, "Choices: ": options, "Correct": correct})

In [ ]:
quiz_df = pd.DataFrame(quiz_table_data)

In [ ]:
quiz_df.to_csv('machine-learning.csv', index=Fal                                                                                                                                                                                                           se)